In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark


tar: spark-2.4.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col,lit
from pyspark.sql.types import  StructField, StructType, StringType, IntegerType,FloatType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler,MinMaxScaler
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName("Modelo de Regreção").getOrCreate()
dataset = spark.read.csv("sample_data/california_housing_train.csv",header=True)
dataset2 = spark.read.csv("sample_data/california_housing_test.csv",header=True)

In [ ]:
dataset.show()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|    

In [ ]:
dataset.count()

17000

In [ ]:
dataset2.count()

3000

In [ ]:
dataset = dataset.union(dataset2)

In [ ]:
dataset.count()

20000

In [ ]:
dataset.show()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|    

In [ ]:
dataset.printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- total_rooms: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- median_house_value: string (nullable = true)



In [ ]:
dataset = dataset.withColumn("longitude",dataset.longitude.cast("Float"))
dataset.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housing_median_age: string (nullable = true)
 |-- total_rooms: string (nullable = true)
 |-- total_bedrooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- median_income: string (nullable = true)
 |-- median_house_value: string (nullable = true)



In [ ]:
dataset.show()

+---------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+---------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  -114.31|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|  -114.47|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|  -114.56|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|  -114.57|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|  -114.57|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|      65500.000000|


In [ ]:
dataset.select("latitude").show)()

NameError: ignored

In [ ]:
lista_float = ["longitude","latitude","median_income","median_house_value"]
dataset = dataset.select([col(c).cast("Float") if c in lista_float else col(c).cast("Integer") for c in dataset.columns ])
dataset.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: float (nullable = true)
 |-- median_house_value: float (nullable = true)



In [ ]:
dataset.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|                15|       5612|          1283|      1015|       472|       1.4936|           66900.0|
|  -114.47|    34.4|                19|       7650|          1901|      1129|       463|         1.82|           80100.0|
|  -114.56|   33.69|                17|        720|           174|       333|       117|       1.6509|           85700.0|
|  -114.57|   33.64|                14|       1501|           337|       515|       226|       3.1917|           73400.0|
|  -114.57|   33.57|                20|       1454|           326|       624|       262|        1.925|           65500.0|
|  -114.58|   33.63|    

In [ ]:
dataset.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in dataset.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+



In [ ]:
vetor_col = "corr_features"
ass = VectorAssembler(inputCols=dataset.columns, outputCol=vetor_col)
df_vetor = ass.transform(dataset).select(vetor_col)
matrix = Correlation.corr(df_vetor, vetor_col)

In [ ]:
df_vetor.show()

+--------------------+
|       corr_features|
+--------------------+
|[-114.30999755859...|
|[-114.47000122070...|
|[-114.55999755859...|
|[-114.56999969482...|
|[-114.56999969482...|
|[-114.58000183105...|
|[-114.58000183105...|
|[-114.58999633789...|
|[-114.58999633789...|
|[-114.59999847412...|
|[-114.59999847412...|
|[-114.59999847412...|
|[-114.61000061035...|
|[-114.61000061035...|
|[-114.62999725341...|
|[-114.65000152587...|
|[-114.65000152587...|
|[-114.65000152587...|
|[-114.66000366210...|
|[-114.66999816894...|
+--------------------+
only showing top 20 rows



In [ ]:
matrix = matrix.collect()[0][0].values
matrix_cor = np.reshape(matrix,(9,9))

In [ ]:

schema = StructType([
    StructField('longitude', FloatType(), True),
    StructField('latitude',  FloatType(), True),
    StructField('housing_median_age',  FloatType(), True),
    StructField('total_rooms',  FloatType(), True),
    StructField('total_bedrooms',  FloatType(), True),
    StructField('population',  FloatType(), True),
    StructField('households',  FloatType(), True),
    StructField('median_income',  FloatType(), True),
    StructField('median_house_value',  FloatType(), True)
])

rdd = spark.sparkContext.parallelize(matrix_cor.tolist())
df = spark.createDataFrame(rdd,schema)
df.show()

+------------+------------+------------------+-----------+--------------+------------+-----------+-------------+------------------+
|   longitude|    latitude|housing_median_age|total_rooms|total_bedrooms|  population| households|median_income|median_house_value|
+------------+------------+------------------+-----------+--------------+------------+-----------+-------------+------------------+
|         1.0| -0.92517567|       -0.10682448|0.047465634|    0.07169962|  0.10299369| 0.05844283| -0.015882086|      -0.045788176|
| -0.92517567|         1.0|       0.010260145|-0.03891092|   -0.06921718| -0.11202547| -0.0739602| -0.079157405|       -0.14396892|
| -0.10682448| 0.010260145|               1.0|-0.36203358|   -0.32087323| -0.29628474| -0.3031087|  -0.12016073|        0.10446951|
| 0.047465634| -0.03891092|       -0.36203358|        1.0|     0.9297782|  0.85682565|   0.918234|   0.19920687|        0.13529752|
|  0.07169962| -0.06921718|       -0.32087323|  0.9297782|           1.0|   

In [ ]:
dataset.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

In [ ]:
assembler = VectorAssembler(inputCols=['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 ], outputCol="features")
df = assembler.transform(dataset)
df = df.withColumn("label",df.median_house_value)


In [ ]:
df = df.drop("median_house_value")
#df = df.drop("longitude")
#df = df.drop("latitude")

In [ ]:
df.groupby("total_rooms").count().show()

+-----------+-----+
|total_rooms|count|
+-----------+-----+
|       1088|    6|
|       7240|    1|
|       2366|    7|
|       1645|    5|
|        833|    5|
|       1580|   12|
|       1591|   11|
|       2866|    4|
|       7880|    2|
|       1342|    5|
|       2142|    9|
|       4935|    1|
|       1959|    6|
|       1829|   10|
|       1238|    3|
|       2122|    7|
|       6336|    1|
|        471|    2|
|       2659|    2|
|       3175|    3|
+-----------+-----+
only showing top 20 rows



In [ ]:
df = df.withColumn('median_income',df.median_income * 1000)
df = df.withColumn('median_income',df.median_income.cast("Integer"))
df = df.withColumn('label',df.label.cast('Integer'))

In [ ]:
df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
df = df.filter(df['label']<300000)

In [ ]:
df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------------------+-----+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|            features|label|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------------------+-----+
|  -114.31|   34.19|                15|       5612|          1283|      1015|       472|         1493|[-114.30999755859...|66900|
|  -114.47|    34.4|                19|       7650|          1901|      1129|       463|         1820|[-114.47000122070...|80100|
|  -114.56|   33.69|                17|        720|           174|       333|       117|         1650|[-114.55999755859...|85700|
|  -114.57|   33.64|                14|       1501|           337|       515|       226|         3191|[-114.56999969482...|73400|
|  -114.57|   33.57|                20|       1454|           326|       624|       262|  

In [ ]:
featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures").fit(df)

In [ ]:
(treino, teste) = df.randomSplit([0.7, 0.3])

dt = GBTRegressor(featuresCol="scaledFeatures")

pipeline = Pipeline(stages=[featureScaler, dt])

model = pipeline.fit(treino)

pred = model.transform(teste)

In [ ]:
pred.select("prediction", "label", "features").show(15)

avaliador = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")

mae = avaliador.evaluate(pred)

print("Erro medio absoluto nos dados de teste = %g" % mae)

GBT = model.stages[1]
print(GBT)

+------------------+------+--------------------+
|        prediction| label|            features|
+------------------+------+--------------------+
| 83707.43521846783| 85800|[-124.30000305175...|
| 95724.24343764603| 79000|[-124.26999664306...|
|106144.92990711823|106700|[-124.23000335693...|
|102653.65407646306| 50800|[-124.23000335693...|
|   96209.929696267| 58100|[-124.20999908447...|
| 98805.54177811905| 69000|[-124.19000244140...|
|  81041.4109249334| 70000|[-124.19000244140...|
|102855.47793960074|107000|[-124.18000030517...|
|    85919.56007191| 70200|[-124.18000030517...|
| 96997.40435467602| 82800|[-124.16999816894...|
|  89556.4743892419| 75500|[-124.16999816894...|
| 93409.56771491795| 85400|[-124.16000366210...|
| 88208.91961847899| 76900|[-124.16000366210...|
| 94743.56145102641| 92700|[-124.16000366210...|
| 95034.42481360912| 74100|[-124.16000366210...|
+------------------+------+--------------------+
only showing top 15 rows

Erro medio absoluto nos dados de teste = 28